# Capstone Project Assignment-5 Course-9

## Opening New Casino/Lounge in Kuala Lumpur, Malaysia

## Importing required libraries

In [1]:
import numpy as np 

import pandas as pd 
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import requests 
from bs4 import BeautifulSoup  

from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors
!pip install geocoder
import geocoder 
from geopy.geocoders import Nominatim
 
from sklearn.cluster import KMeans
import json
!pip install folium
import folium # Library to render maps

print("All required libraries has been imported...")

All required libraries has been imported...


## Let's scrap the neighbourhood data from the wiki page

In [2]:
wikiData = requests.get("https://en.wikipedia.org/wiki/Category:Suburbs_in_Kuala_Lumpur").text

### Parse the wikidata into a soup object

In [3]:
soupObject = BeautifulSoup(wikiData, 'html.parser')

In [4]:
neighbourhoodDataList = [] # List to contain the neighbourhood data
for row in soupObject.find_all("div", class_="mw-category")[0].findAll("li"):
    neighbourhoodDataList.append(row.text)
    

### Let's create a dataframe to store neighbourhood from the list we created

In [5]:
neighbourhood_df = pd.DataFrame({"Neighborhood": neighbourhoodDataList})
neighbourhood_df.head()

,Neighborhood
0,Alam Damai
1,"Ampang, Kuala Lumpur"
2,Bandar Menjalara
3,Bandar Sri Permaisuri
4,Bandar Tasik Selatan


In [6]:
neighbourhood_df.shape

(71, 1)

### Finding out the geographical coordinates of the neighbourhoods 

In [11]:
# define a function to get coordinates
def find_coords(neighborhood):
   
    lat_long_coords = None
    
    while(lat_long_coords is None):
        g = geocoder.arcgis('{}, Kuala Lumpur, Malaysia'.format(neighborhood))
        lat_long_coords = g.latlng
    return lat_long_coords

In [12]:
# calling the function and extracting the coords
coordinates_neighbourhood = [ find_coords(neighborhood) for neighborhood in neighbourhood_df["Neighborhood"].tolist() ]

In [13]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coordinates = pd.DataFrame(coordinates_neighbourhood, columns=['Latitude', 'Longitude'])

In [14]:
df_coordinates.head()

,Latitude,Longitude
0,3.057690,101.743880
1,3.148499,101.696728
2,3.190350,101.625450
3,3.103910,101.712260
4,3.072750,101.714610


### Let's merge the coordinates into the original dataframe

In [17]:
neighbourhood_df['Latitude'] = df_coordinates['Latitude']
neighbourhood_df['Longitude'] = df_coordinates['Longitude']

In [18]:
# check the neighborhoods and the coordinates
print(neighbourhood_df.shape)
neighbourhood_df

(71, 3)


,Neighborhood,Latitude,Longitude
0,Alam Damai,3.057690,101.743880
1,"Ampang, Kuala Lumpur",3.148499,101.696728
2,Bandar Menjalara,3.190350,101.625450
3,Bandar Sri Permaisuri,3.103910,101.712260
4,Bandar Tasik Selatan,3.072750,101.714610
5,Bandar Tun Razak,3.082800,101.722810
6,Bangsar,3.129200,101.678440
7,Bangsar Park,3.129200,101.678440
8,Bangsar South,3.111020,101.662830
9,Batu 11 Cheras,3.061870,101.746750


## Let's create the map for the Kuala Lumpur, Malaysia

### Let's first find out the coords of the city

In [19]:
# get the coordinates of Kuala Lumpur
address = 'Kuala Lumpur, Malaysia'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Kuala Lumpur, Malaysia {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Kuala Lumpur, Malaysia 3.1516964, 101.6942371.


In [20]:
# create map of Toronto using latitude and longitude values
city_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(neighbourhood_df['Latitude'], neighbourhood_df['Longitude'], neighbourhood_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(city_map)  
    
city_map

### Now, Let's use the FourSquare APIs to extact the venues data of the Kuala Lumpur, Malaysia Neighbourhoods

In [59]:
# Let's set up the credentials for the FourSquare APIs
CLIENT_ID = 'CLIENT_ID' # your Foursquare ID
CLIENT_SECRET = 'CLIENT_SECRET' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CLIENT_ID
CLIENT_SECRET:CLIENT_SECRET


### Let's extract the top 100 venues within 2KM radius

In [26]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(neighbourhood_df['Latitude'], neighbourhood_df['Longitude'], neighbourhood_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

#### Let's create a dataframe for the venues received from FourSqaure APIs

In [28]:
venues_frame = pd.DataFrame(venues)

# define the column names
venues_frame.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_frame.shape)
venues_frame.head()

(7086, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Alam Damai,3.05769,101.74388,Jc Deli 皆喜食坊,3.058397,101.748560,Food & Drink Shop
1,Alam Damai,3.05769,101.74388,Pengedar Shaklee Kuala Lumpur,3.061235,101.740696,Supplement Shop
2,Alam Damai,3.05769,101.74388,Machi Noodle 妈子面,3.057695,101.746635,Noodle House
3,Alam Damai,3.05769,101.74388,628火焰鑫茶室,3.058442,101.747947,Chinese Restaurant
4,Alam Damai,3.05769,101.74388,Roti Canai Perindu,3.055071,101.740574,Breakfast Spot


### Let's check the Venues returned for the neighbourhoods

In [29]:
venues_frame.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Alam Damai,100,100,100,100,100,100
"Ampang, Kuala Lumpur",100,100,100,100,100,100
Bandar Menjalara,100,100,100,100,100,100
Bandar Sri Permaisuri,100,100,100,100,100,100
Bandar Tasik Selatan,100,100,100,100,100,100
Bandar Tun Razak,100,100,100,100,100,100
Bangsar,100,100,100,100,100,100
Bangsar Park,100,100,100,100,100,100
Bangsar South,100,100,100,100,100,100


### Checking for the total unique categories

In [30]:
print(' {} uniques categories.'.format(len(venues_frame['VenueCategory'].unique())))

 314 uniques categories.


#### Let's check if we are getting Casinos information

In [32]:
"Casino" in venues_frame['VenueCategory'].unique()

True

## Let's create a dataframe consisting all neighbourhoods

In [33]:
# one hot encoding
onehot_encoding = pd.get_dummies(venues_frame[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot_encoding['Neighborhoods'] = venues_frame['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [onehot_encoding.columns[-1]] + list(onehot_encoding.columns[:-1])
onehot_encoding = onehot_encoding[fixed_columns]

print(onehot_encoding.shape)
onehot_encoding.head()

(7086, 315)


,Neighborhoods,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Business Service,Butcher,Cafeteria,Café,Camera Store,Campground,Candy Store,Cantonese Restaurant,Capitol Building,Casino,Chettinad Restaurant,Chinese Breakfast Place,Chinese Restaurant,Circus,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dive Shop,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Event Space,Exhibit,Fabric Shop,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hainan Restaurant,Hakka Restaurant,Halal Restaurant,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Hill,Historic Site,History Museum,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kushikatsu Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Lounge,Malay Restaurant,Mamak Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Event Space,Outlet Store,Pakistani Restaurant,Palace,Park,Performing Arts Venue,Pet Café,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Print Shop,Pub,Public Art,Racetrack,Ramen Restaurant,Recording Studio,Recreation Center,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Street Art,Street Food Gathering,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Re

### Let's check avg frequency for the venues by grouping them on neighbourhoods

In [34]:
neighbourhoods_grouped = onehot_encoding.groupby(["Neighborhoods"]).mean().reset_index()

print(neighbourhoods_grouped.shape)
neighbourhoods_grouped

(71, 315)


,Neighborhoods,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Business Service,Butcher,Cafeteria,Café,Camera Store,Campground,Candy Store,Cantonese Restaurant,Capitol Building,Casino,Chettinad Restaurant,Chinese Breakfast Place,Chinese Restaurant,Circus,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dive Shop,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Event Space,Exhibit,Fabric Shop,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hainan Restaurant,Hakka Restaurant,Halal Restaurant,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Hill,Historic Site,History Museum,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kushikatsu Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Lounge,Malay Restaurant,Mamak Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Event Space,Outlet Store,Pakistani Restaurant,Palace,Park,Performing Arts Venue,Pet Café,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Print Shop,Pub,Public Art,Racetrack,Ramen Restaurant,Recording Studio,Recreation Center,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Street Art,Street Food Gathering,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Re

### Let's check for the # of casinos/Lounges in the city

In [44]:
len(neighbourhoods_grouped[neighbourhoods_grouped["Casino"] > 0])

1

In [45]:
len(neighbourhoods_grouped[neighbourhoods_grouped["Lounge"] > 0])

22

## Creating a new dataframe consisting of Casino/Lounges Data

In [46]:
cl_df = neighbourhoods_grouped[["Neighborhoods","Casino","Lounge"]]

In [47]:
cl_df.head()

,Neighborhoods,Casino,Lounge
0,Alam Damai,0.00,0.00
1,"Ampang, Kuala Lumpur",0.00,0.00
2,Bandar Menjalara,0.00,0.01
3,Bandar Sri Permaisuri,0.00,0.00
4,Bandar Tasik Selatan,0.01,0.00


## Let's use K-means Clustering on our dataframe

### Let's run it for 3 clusters in the Kuala Lumpur city

In [48]:
# setting number of clusters
kclusters = 3

kl_clustering = cl_df.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 1, 0, 0, 0, 1, 1, 0, 0], dtype=int32)

## Let's create a new Dataset consisting of clusters info as well

In [49]:

cl_merged = cl_df.copy()

# add clustering labels
cl_merged["Cluster Labels"] = kmeans.labels_

In [50]:
cl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
cl_merged.head()

,Neighborhood,Casino,Lounge,Cluster Labels
0,Alam Damai,0.00,0.00,0
1,"Ampang, Kuala Lumpur",0.00,0.00,0
2,Bandar Menjalara,0.00,0.01,1
3,Bandar Sri Permaisuri,0.00,0.00,0
4,Bandar Tasik Selatan,0.01,0.00,0


### Merge the Lat-Long details with the existing dataframe

In [51]:
cl_merged = cl_merged.join(neighbourhood_df.set_index("Neighborhood"), on="Neighborhood")

print(cl_merged.shape)
cl_merged.head()

(71, 6)


,Neighborhood,Casino,Lounge,Cluster Labels,Latitude,Longitude
0,Alam Damai,0.00,0.00,0,3.057690,101.743880
1,"Ampang, Kuala Lumpur",0.00,0.00,0,3.148499,101.696728
2,Bandar Menjalara,0.00,0.01,1,3.190350,101.625450
3,Bandar Sri Permaisuri,0.00,0.00,0,3.103910,101.712260
4,Bandar Tasik Selatan,0.01,0.00,0,3.072750,101.714610


In [52]:
# sort the results by Cluster Labels
print(cl_merged.shape)
cl_merged.sort_values(["Cluster Labels"], inplace=True)
cl_merged

(71, 6)


,Neighborhood,Casino,Lounge,Cluster Labels,Latitude,Longitude
0,Alam Damai,0.00,0.000000,0,3.057690,101.743880
34,Kepong,0.00,0.000000,0,3.217500,101.637630
69,Titiwangsa,0.00,0.000000,0,3.180730,101.703210
36,Kuchai Lama,0.00,0.000000,0,3.090740,101.677330
37,Lembah Pantai,0.00,0.000000,0,3.121189,101.663889
38,Maluri,0.00,0.000000,0,3.147890,101.694050
40,Miharja,0.00,0.000000,0,3.147890,101.694050
41,Mont Kiara,0.00,0.000000,0,3.165320,101.652430
42,Pantai Dalam,0.00,0.000000,0,3.094760,101.667470
44,Putrajaya,0.00,0.000000,0,3.125851,101.718513


## Drawing the map with the clusters

In [53]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(cl_merged['Latitude'], cl_merged['Longitude'], cl_merged['Neighborhood'], cl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [54]:
# saving map as the HTMl File
map_clusters.save('map_clusters.html')

### Let's examine our resultant clusters finally

### Cluster-0

In [55]:
cl_merged.loc[cl_merged['Cluster Labels'] == 0]

,Neighborhood,Casino,Lounge,Cluster Labels,Latitude,Longitude
0,Alam Damai,0.00,0.0,0,3.057690,101.743880
34,Kepong,0.00,0.0,0,3.217500,101.637630
69,Titiwangsa,0.00,0.0,0,3.180730,101.703210
36,Kuchai Lama,0.00,0.0,0,3.090740,101.677330
37,Lembah Pantai,0.00,0.0,0,3.121189,101.663889
38,Maluri,0.00,0.0,0,3.147890,101.694050
40,Miharja,0.00,0.0,0,3.147890,101.694050
41,Mont Kiara,0.00,0.0,0,3.165320,101.652430
42,Pantai Dalam,0.00,0.0,0,3.094760,101.667470
44,Putrajaya,0.00,0.0,0,3.125851,101.718513


### Cluster-1

In [57]:
cl_merged.loc[cl_merged['Cluster Labels'] == 1]

,Neighborhood,Casino,Lounge,Cluster Labels,Latitude,Longitude
2,Bandar Menjalara,0.0,0.010000,1,3.190350,101.625450
64,Taman Sri Sinar,0.0,0.010101,1,3.190070,101.652930
27,"Jalan Cochrane, Kuala Lumpur",0.0,0.010000,1,3.134630,101.721690
6,Bangsar,0.0,0.010000,1,3.129200,101.678440
7,Bangsar Park,0.0,0.010000,1,3.129200,101.678440
25,"Federal Hill, Kuala Lumpur",0.0,0.010000,1,3.136370,101.685640
24,Desa Petaling,0.0,0.010000,1,3.083300,101.704380
43,"Pudu, Kuala Lumpur",0.0,0.010000,1,3.133540,101.713070
45,Salak South,0.0,0.010000,1,3.081540,101.696890
55,Taman Bukit Maluri,0.0,0.010000,1,3.200660,101.633370


### Cluster-2

In [58]:
cl_merged.loc[cl_merged['Cluster Labels'] == 2]

,Neighborhood,Casino,Lounge,Cluster Labels,Latitude,Longitude
12,Bukit Bintang,0.0,0.03,2,3.14777,101.70855
14,Bukit Kiara,0.0,0.02,2,3.14348,101.64433
67,Taman U-Thant,0.0,0.02,2,3.15770,101.72452


### Observations

We have clustered our data on to the Casino and the Lounges. On our data, we found that the city has only casino but the fair number of Lounges.
The Cluster-0 shows us that there is a high chances of opening new Lounges there with no existing competition and only having a single Casino.
The Cluster-1 shows us that the there is no casino in those areas but fair number of Lounges are present so there is a fair competition.
The Cluster-2 shows us that the Lounges are comparatively high and chances of the competition is higher.